In [2]:
## importing necessary packages
import pandas as pd
import numpy as np
import os
import holidays
pd.set_option("display.max_columns",None)

In [62]:
## readiing the yearwise ride data
rides_data = os.listdir(".\Yearwise_Files")

In [126]:
def ride_data_grouping(file):
    data = pd.read_csv(os.path.join(".\Yearwise_Files",file),usecols=["start_date","member_type"])
    # converting "start_date" variable to datetime type
    data["start_date"] = pd.to_datetime(data["start_date"])
    data["date"] = data["start_date"].dt.date    # Extracting date part from "start_date"
    data["hour"] = data["start_date"].dt.hour    # Extracting hour part from "start_date"
    data = data.drop(columns = "start_date")  
    
    # converting the values of "member_type" variable to lower case
    data["member_type"] = data["member_type"].apply(lambda x: x.lower())
    
    # Grouping the data according to member_type and finding the count of each type for each day and hour
    data2 = data.groupby(by = ["date","hour","member_type"]).value_counts().reset_index()
    
    # Creating different datasets for different member_type
    data2_casual = data2[data2["member_type"]=="casual"]
    data2_member = data2[data2["member_type"]=="member"]
    
    # Merging the above datasets according to date and hour, to get count of Casual and Member riders for each hour and day
    data3 = data2_casual.merge(data2_member, on = ["date","hour"],how="outer")
    # Renaming the columns of merged dataset
    data3.rename(columns = {"0_x":"casual","0_y":"member"},inplace = True)
    # Arranging the columns for better understanding
    data3 = data3.loc[:,["date","hour","casual","member"]]
    # Sorting the merged dataset  according to date then hour
    data3.sort_values(by=["date","hour"],inplace = True)
    
    # There are some NaN values after merging, replacing those with 0
    data3["casual"] = data3["casual"].fillna(0)
    data3["member"] = data3["member"].fillna(0)
    
    # Calculating total number of riders for each hour in a day
    data3["total_count"] = data3["casual"]+data3["member"]
    return data3

In [127]:
## Checking whether the day is holiday or not
def holiday_check(date):
    us_holidays = holidays.US(state="DC")
    if us_holidays.get(date) == None:
        return 0
    else:
        return 1

# Getting the season of the year for the passed month
def get_season(month):
    if month in [1,2,3]:
        return "winter"
    elif month in [4,5,6]:
        return "spring"
    elif month in [7,8,9]:
        return "summer"
    else:
        return "fall"

In [130]:
# Creating directory to store .CSV files 
os.makedirs("Final_Ride_Data",exist_ok=True)
for file in rides_data:
    df = ride_data_grouping(file)
    df["date"] = pd.to_datetime(df["date"])
    ## Extracting month, year, weekday from the date variable
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year
    df["weekday"] = df["date"].dt.weekday
    
    ## Creating the "is_holiday" variable to capture whether the day is holiday(1) or not(0) 
    df["is_holiday"] = df["date"].apply(holiday_check)
    
    ## Creating the "working_day" variable to capture whether the day is working day(1) or not(0) 
    df["working_day"] = np.where((df["is_holiday"]==1),0,(np.where((df["weekday"]>=5),0,1)))
    
    ## getting season for the month
    df["season"] = df["month"].apply(get_season)
    ## Arranging the columns for better understanding
    df = df.loc[:,["date","year","month","hour","season","weekday","is_holiday","working_day","casual","member","total_count"]]
    ## Saving the dataset .CSV file
    df.to_csv(f".\Final_Ride_Data/ridedata_{df['year'][0]}.csv",index = None)
    print(f"Successfully derived ride data for year: {df['year'][0]}")

Successfully derived ride data for year: 2018
Successfully derived ride data for year: 2019
Successfully derived ride data for year: 2020
Successfully derived ride data for year: 2021
Successfully derived ride data for year: 2022


## Ride and Weather Data Combine

In [3]:
final_ride = os.listdir(".\Final_Ride_Data")
weather_data = os.listdir(".\Weather_Hourly_Data")

In [4]:
final_ride

['ridedata_2018.csv',
 'ridedata_2019.csv',
 'ridedata_2020.csv',
 'ridedata_2021.csv',
 'ridedata_2022.csv']

In [5]:
weather_data

['2018-weather-data.csv',
 '2019-weather-data.csv',
 '2020-weather-data.csv',
 '2021-weather-data.csv',
 '2022-weather-data.csv']

In [25]:
os.makedirs(".\Final_Derived_Data")
for i in range(len(final_ride)):
    ride = pd.read_csv(".\Final_Ride_Data/"+final_ride[i])
    # converting "date" variable to datetime type
    ride["date"] = pd.to_datetime(ride["date"])     
    
    weather = pd.read_csv(".\Weather_Hourly_Data/"+weather_data[i])
    # Dropping the unwanted variables
    weather.drop(columns= ["Wind Gust","Dew Point","Pressure"],inplace=True)
    # Renaming the columns
    weather.columns = ["date", "time", "temperature(in celcius)", "relative_temperature(in celcius)", 
                       "wind_speed(in kmph)","humidity(in %)", "description"]
    # converting "date" variable to datetime type
    weather["date"] = pd.to_datetime(weather["date"])
    # Extracting "hour" part from time variable
    weather["hour"] = pd.to_datetime(weather["time"], format="%H:%M").dt.hour
    weather.drop(["time"],axis=1, inplace=True)
    
    # Merging the ride data and weather data for each year
    derived_data = ride.merge(weather, on = ["date","hour"])
    derived_data.to_csv(f".\Final_Derived_Data/final_data_{derived_data['year'][0]}.csv",index=None)
    print(f"Successfully derived data for year: {derived_data['year'][0]}")

Successfully derived data for year: 2018
Successfully derived data for year: 2019
Successfully derived data for year: 2020
Successfully derived data for year: 2021
Successfully derived data for year: 2022
